In [ ]:
import pandas as pd
import numpy as np
import ast
import gzip
import json
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

<h1>STEAM GAMES</h1>

In [ ]:
#Carga de DataSet steam_games
steam_games=pd.read_json('../Datasets/steam_games.json.gz',compression='gzip',lines=True)


In [ ]:

# Elimina las filas donde todos sus valores son nulos
steam_games.dropna(how='all',inplace=True)


In [ ]:
#Proceso de Limpieza Columna 'id'
steam_games[steam_games['id'].isnull()]


In [ ]:
steam_games[steam_games['title'] == 'Batman: Arkham City - Game of the Year Edition'] 

In [ ]:
#Se puede observar que se encontraron dos filas con id nuelos.
#El primero carece de gran parte de su informacion, por tanto sera eliminada.
#El segundo se puede concluir que se trata de una fila duplicada donde una de 
#esta no tiene el id, por lo que sera eliminada

steam_games.dropna(subset=['id'],inplace=True)


In [ ]:

#Busqueda de duplicados
duplicates = steam_games[steam_games.duplicated(subset=['id'], keep=False)]
print(duplicates)


In [ ]:
#Se puede observar dos filas duplicada, por tanto se elimina una de estas
steam_games.drop_duplicates(subset=['id'], keep='first', inplace=True)

In [ ]:
#Convertir los valores de la columna 'id' en enteros
steam_games['id'] = steam_games['id'].astype(int)
steam_games.info()


In [ ]:
steam_games.isnull().sum()

In [ ]:
#Proceso de Limpieza Columna 'app_name'

#Busqueda de app_name nulos
steam_games[steam_games['app_name'].isnull()]


In [ ]:
#Se encontro una appe_nume nula la cual carece de otras informaciones imnportantes,
# por lo que sera eliminada
steam_games.dropna(subset=['app_name'],inplace=True)

In [ ]:
#Proceso de Limpieza Columna 'developer'
#Se remplazara los nulos de developer por los de publisher
steam_games['developer'].fillna(steam_games['publisher'],inplace=True)


In [ ]:
#Proceso de limpieza columna 'genres'
#Los valores de tags son iguales a genres, cargare los fataltante de genres con tags, 
#y creo un conjunto (set) con totos valores unicos de genres.

#Creando un conjunto ( set) que contiene todos los 
#géneros únicos presentes en la columna 'géneros' del DataFrame
genres = set(item for val in steam_games['genres'].dropna() for item in val)

In [ ]:
#Filtrando las etiquetas 'tags' para mantener solo aquellas 
#que están presentes en 'genres'
steam_games['tags'] = steam_games['tags'].apply(lambda x: [item for item in x if item in genres] if isinstance(x, list) else x) 


In [ ]:
#Carga de valores nulos en la columna 'genres' con valores de 'tags'
steam_games['genres'].fillna(steam_games['tags'], inplace=True)


In [ ]:
#Funcion que añade valores a 'geners' que estan en 'tags'
 #Función que añade valores de 'tags' a 'genres' que no estén ya presentes en 'genres'
def agregar_genres(fila):
    genres = fila['genres']
    tags = fila['tags']
    if isinstance(tags,list) and isinstance(genres,list):
        for tag in tags:
            if tag not in genres:
                genres.append(tag)
    return genres


In [ ]:
#Aplicando la función 'agregar_genres' a todos los elementos en el dataframe
steam_games['genres'] = steam_games.apply(lambda fila: agregar_genres(fila), axis=1)

In [ ]:
#Creacion de columnas ficticias a travez d dumies con los valores de geners
# Ahora voy a generar dummies para genres 
steam_games['genres'] =  steam_games['genres'].apply(lambda x:".".join(x) if isinstance(x, list) else x)
dummies = steam_games['genres'].str.get_dummies(sep='.')
dummies = dummies.groupby(dummies.columns, axis=1).sum()
steam_games = pd.concat([steam_games,dummies],axis=1)
steam_games.head(2)

In [ ]:

#### Elimino todas las columnas inecesarias
# publisher
# genres
# title
# url
# tags
# discount_price
# reviews_url
# early_access
# specs
# price


steam_games = steam_games.drop(columns=['publisher', 'genres', 'title', 'url', 'tags', 'reviews_url', 'early_access', 'specs', 'price'])

In [ ]:
steam_games.isnull().sum()

In [ ]:
#Evaluacion de Porcentaje de nulos
print(f'Porcentaje de nulos : {1 - steam_games.dropna(subset=["developer","release_date"]).shape[0] / steam_games.shape[0]}')

In [ ]:
#Entre las columnas release_date y developer hacen un 11% de valores nulos. al poder 
#hacer mas analisis y asignar valores, procedo a la eliminacion de dichas filas
steam_games.dropna(subset=['developer','release_date'],inplace=True)
steam_games.isnull().sum()

In [ ]:
print(steam_games.info())

In [ ]:
#Proceso de Limpieza Columna release_date
steam_games['release_date'].value_counts()


In [ ]:
#Verifico el formato de los año usando re
# Función para verificar el formato 'YYYY-MM-DD'
def is_valid_date_format(date_str):
    regex = r'^\d{4}-\d{2}-\d{2}$'
    return re.match(regex, date_str) is not None

# Crear una máscara booleana para las fechas que no tienen el formato 'YYYY-MM-DD'
invalid_date_mask = ~steam_games['release_date'].astype(str).apply(is_valid_date_format)

# Filtrar las filas con fechas que no tienen el formato 'YYYY-MM-DD'
invalid_dates = steam_games[invalid_date_mask]

# Mostrar las fechas que no tienen el formato 'YYYY-MM-DD'
print(invalid_dates['release_date'])

In [ ]:
#Hago una funcion para tratar los valores que no tienen el formato YYYY-MM-DD
def arreglar_errores(fecha: str):
    try:
        # Intenta convertir la fecha directamente a entero
        return int(fecha)
    except ValueError:
        try:
            # Intenta dividir la fecha usando un espacio en blanco como delimitador y convierte la segunda parte a entero
            return int(fecha.split(" ")[1])
        except (IndexError, ValueError):
            try:
                # Intenta dividir la fecha usando "." como delimitador y convierte la tercera parte a entero
                return int(fecha.split(".")[2])
            except (IndexError, ValueError):
                try:
                    # Intenta dividir la fecha usando un espacio en blanco como delimitador y convierte la quinta parte a entero
                    return int(fecha.split(" ")[4])
                except (IndexError, ValueError):
                    # Si no se puede convertir, devuelve None
                    return None

In [ ]:
steam_games.loc[invalid_date_mask, 'Year'] = steam_games.loc[invalid_date_mask, 'release_date'].apply(arreglar_errores)

# Convertir las fechas válidas a datetime y extraer el año
valid_date_mask = ~invalid_date_mask
steam_games.loc[valid_date_mask, 'Year'] = pd.to_datetime(steam_games.loc[valid_date_mask, 'release_date'], errors='coerce').dt.year

In [ ]:
#llevo la columna Year a formato int
steam_games['Year'] = steam_games['Year'].astype('Int64')

In [ ]:
print(steam_games.info())

In [ ]:
#Elimino la Columna 'release_date'
steam_games.drop(columns=['release_date'],inplace=True)

In [ ]:
#Elimino las filas con valores vacio de la columna 'Year'
steam_games['Year'].isnull().sum()
steam_games['Year'].replace('', np.nan, inplace=True)
steam_games.dropna(subset=['Year'], inplace=True)

In [ ]:
#elimino la columna developer
steam_games = steam_games.drop(columns=['developer'])

In [ ]:
#trabajo con el 10% de la base de datos
porcentaje_a_eliminar = 0.90

# Calcula el número de filas a eliminar
num_filas_a_eliminar = int(len(steam_games) * porcentaje_a_eliminar)

# Utiliza el método sample para seleccionar aleatoriamente las filas a eliminar
filas_a_eliminar = steam_games.sample(n=num_filas_a_eliminar, random_state=42)

# Elimina las filas seleccionadas del DataFrame original
steam_games = steam_games.drop(filas_a_eliminar.index)

In [ ]:
steam_games.to_csv('../Datasets/steam_games.csv',index=False)

<h1>USERS REVIEWS</h1>

In [ ]:
#Carga de DataSet Orignal
user_reviews_gz = "../Datasets/user_reviews.json.gz"
filas=[]
with gzip.open(user_reviews_gz, 'rt', encoding='MacRoman') as archivo:
    for line in archivo.readlines():
        filas.append(ast.literal_eval(line))

user_review = pd.DataFrame(filas)

In [ ]:
user_review .head()

El Datasets user_review contiene 3 columnas donde la 'reviews' es una lista de diccionario que contiene iformacion de los comentarios realizados en cada juego.

por consiguiente se procede al desanidado de la columna'reviews' repitiendo las filas del usuario para cada item_id.

In [ ]:
user_review_explode = user_review.explode('reviews') # Duplico las filas generando un diccionario por cada diccionario en la lista
# Ahora concateno el dataframe original, con el dataframe generado a partir de transformar los diccionarios a pandas
user_review_explode = pd.concat([user_review_explode.drop(['reviews'],axis=1),user_review_explode['reviews'].apply(pd.Series)],axis=1)  

### Elimino columnas que no seran usadas para el estudio
* user_url
* funny
* last_edited
* helpful
* 0

In [ ]:
user_review_explode.drop(columns=['user_url','funny','helpful','last_edited',0],inplace=True)

In [ ]:
user_review_explode.head(1)

<h5> A partir de la columna reveiw se hara un analisis de sentimiento, generando una nueva columna.
 Este se llamara "sentiment_analysis"  y tendra 2 para sentimiento positivo 0 para negativo y 1 para neutral
 Habiendo hecho esto elimino la columna review </h5>

In [ ]:
#Hago el analisis de sentimiento en la columna review
nltk.download('vader_lexicon')
model_sentimiento = SentimentIntensityAnalyzer()


def analizador(review):
    # Obtener el puntaje de sentimiento usando SentimentIntensityAnalyzer
    sentimiento_score = model_sentimiento.polarity_scores(review)
    
    # Clasifico el sentimiento
    
    if review and not pd.isnull(review):
        if sentimiento_score['compound'] >= 0.5:
            return 2  # Sentimiento positivo
        elif sentimiento_score['compound'] <= -0.5:
            return 0  # Sentimiento negativo
        else:
            return 1  # Sentimiento neutral
    else:
        return 1

* Para aplicar la funcion los nulos tienen que ser vacio

In [ ]:
user_review_explode['review'].fillna('',inplace=True) # Remplazo los nulos con un string vacio

In [ ]:
user_review_explode['sentiment_analysis']  = user_review_explode['review'].apply(analizador)
user_review_explode.drop(columns='review',inplace=True) # Aplica la funcion que determina el sentimiento

* Grafico las el analisis por sentimiento de las reviews

In [ ]:
sns.violinplot(x='sentiment_analysis', data=user_review_explode)
plt.title('Violinplot de Análisis de Sentimientos')
plt.xlabel('Puntuación de Sentimiento')
plt.show()

Muestro la Cantidad de nulos por columna

In [ ]:
user_review_explode.isnull().sum()

In [ ]:
user_review_explode[user_review_explode.isnull().any(axis=1)]

Se eliminan las filas nulas

In [ ]:
user_review_explode.dropna(inplace=True)

In [ ]:
user_review_explode.head()

In [ ]:
#trabajo con el 10% de la base de datos
porcentaje_a_eliminar = 0.90

# Calcula el número de filas a eliminar
num_filas_a_eliminar = int(len(user_review_explode) * porcentaje_a_eliminar)

# Utiliza el método sample para seleccionar aleatoriamente las filas a eliminar
filas_a_eliminar = user_review_explode.sample(n=num_filas_a_eliminar, random_state=42)

# Elimina las filas seleccionadas del DataFrame original
user_review_explode = user_review_explode.drop(filas_a_eliminar.index)

Exporta como csv

In [ ]:
user_review_explode.to_csv('../datasets/user_reviews.csv',index=False)

<h1> USERS ITEMS </h1>

Descomprimo el dataset de items

In [ ]:
def descompimir_json(ruta, variable_anidada):
    '''Función que recibe una ruta de acceso a un archivo json anidado y carga la información en un
    DataFrame de Pandas'''
    fila = []
    with gzip.open(ruta, 'rt', encoding='MacRoman') as archivo:
      for line in archivo.readlines():
          fila.append(ast.literal_eval(line))

    df = pd.DataFrame(fila)                                                 
    df = df.explode(variable_anidada).reset_index()                         
    df = df.drop(columns="index")                                           
    df = pd.concat([df, pd.json_normalize(df[variable_anidada])], axis=1)   
    df = df.drop(columns=variable_anidada)                                  

    return df

In [ ]:
users_items = descompimir_json("../datasets/users_items.json.gz",'items')

In [ ]:
users_items.describe()

In [ ]:
users_items.isnull().sum()

In [ ]:
users_items[users_items['item_id'].isna()]

Todos los nulos hacen referencia a las mismas filas por esto seran eliminados

In [ ]:
users_items.dropna(inplace=True)

### Por ultimo elimino las columnas que no usare
* user_url
* playtime_2weeks
* steam_id
* item_name

In [ ]:
users_items.drop(columns=['user_url','playtime_2weeks','steam_id', 'item_name'], inplace=True)

In [ ]:
sns.boxplot(data=users_items,y='playtime_forever') # Hago un diagrama de caja para ver outliers

Tiene muchos outliers, pero si se consideran minutos no tienen que ser necesariamente errores

Exporto el dataframe a csv y lo comprimo en gz

In [ ]:
users_items.isnull().sum()

In [ ]:
users_items.to_csv('../datasets/users_items.csv',index=False)

In [ ]:
#trabajo con el 10% de la base de datos
porcentaje_a_eliminar = 0.90

# Calcula el número de filas a eliminar
num_filas_a_eliminar = int(len(users_items) * porcentaje_a_eliminar)

# Utiliza el método sample para seleccionar aleatoriamente las filas a eliminar
filas_a_eliminar = users_items.sample(n=num_filas_a_eliminar, random_state=42)

# Elimina las filas seleccionadas del DataFrame original
users_items = users_items.drop(filas_a_eliminar.index)

In [ ]:
users_items.to_csv('../datasets/users_items.csv',index=False)

In [ ]:
with gzip.open('../datasets/users_items_project.csv.gz', 'wb') as f:
    users_items.to_csv(f, index=False, encoding='utf-8')